# Setup

## Make both T4 GPUs visiable to CUDA

In [1]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

## Install vLLM

In [2]:
%%time
!pip uninstall -y torch
!pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Looking in links: /kaggle/input/vllm-whl
Processing /kaggle/input/vllm-whl/vllm-0.4.0.post1-cp310-cp310-manylinux1_x86_64.whl
Processing /kaggle/input/vllm-whl/cmake-3.29.0.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/xformers-0.0.23.post1-cp310-cp310-manylinux2014_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/pynvml-11.5.0-py3-none-any.whl (from vllm)
Processing /kaggle/input/vllm-whl/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/outlines-0.0.34-py3-none-any.whl (from vllm)
Processing /kaggle/input/vllm-whl/tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/interegular-0.3.3-py37-n

## Some Configuration

In [3]:
# In DEBUG mode, infer only on 5 problems
DEBUG = False
# Number of candidate solutions to generate
K = 4
DEPTH = 4
TEMPERATURE = 0.35
TOP_P = 0.775
BATCH_SIZE = 64

## Imports

In [4]:
import vllm
import re
import csv
import torch
import gc
from tqdm import tqdm
import pandas as pd
from queue import Queue, Empty
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager

import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

2024-10-31 13:23:43,771	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Python Code Execution Environment

In [5]:
1

1

## Find Python code blocks within text

In [6]:
def find_python_blocks(text):
    blocks = re.findall(r"(```python.*?```)", text, re.DOTALL)
    # filter blocks by trying to convert them to float or int
    filtered_blocks = []
    for block in blocks:
        code = block[len("```python"):-len("```")].strip()
        try:
            x = int(code)
        except:
            filtered_blocks.append(code)
            continue
        try:
            x = float(code)
        except:
            filtered_blocks.append(code)
    return filtered_blocks        

## Class to Execute Python code (adopted from Numina)

In [7]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout
    # handles timeout
    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()

## Execute a Python code block

In [8]:
def execute(executor, code):
    success = False
    for lib in ("subprocess", "venv"):
        if lib in code:
            output = f"{lib} is not allowed"
            outputs.append(output)
            successes.append(success)
            continue
    try:
        success, output = executor(code)
    except TimeoutError as e:
        output = str(e)

    output = output.strip()
    
    return output, success

## Test by running some python code

In [9]:
text = """Block 1
```python
s = 0
for i in range(100):
    s += i
print(s)
```
Block 2
```python
2**12
```
Block 3
```python
3
```
"""

In [10]:
blocks = find_python_blocks(text)
blocks

['s = 0\nfor i in range(100):\n    s += i\nprint(s)', '2**12']

In [11]:
executor = PythonREPL()
outputs = [execute(executor, block) for block in blocks]
outputs

[('4950', True), ('4096', True)]

In [12]:
for block, output in zip(blocks, outputs):
    print(f"""
```python
{block}
```
```output
{output[0]}
```"""
    )


```python
s = 0
for i in range(100):
    s += i
print(s)
```
```output
4950
```

```python
2**12
```
```output
4096
```


# Load Model on vLLM

## We use the Qwen 2.5 7b Instruct Model here by Alibaba. You should explore other models.

In [13]:
llm = vllm.LLM(
    "Qwen/Qwen2.5-32B-Instruct-AWQ",
    tensor_parallel_size=2, 
    quantization="AWQ",
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=10000,
)
tokenizer = llm.get_tokenizer()

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

WARNING 10-31 13:23:47 config.py:211] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-10-31 13:23:50,265	INFO worker.py:1749 -- Started a local Ray instance.


INFO 10-31 13:23:51 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='Qwen/Qwen2.5-32B-Instruct-AWQ', tokenizer='Qwen/Qwen2.5-32B-Instruct-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=10000, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

INFO 10-31 13:26:31 model_runner.py:104] Loading model weights took 9.0933 GB
(RayWorkerVllm pid=372) INFO 10-31 13:26:31 model_runner.py:104] Loading model weights took 9.0933 GB
INFO 10-31 13:26:49 ray_gpu_executor.py:240] # GPU blocks: 1094, # CPU blocks: 2048


# Utilites

## Extract boxed answer

In [14]:
def extract_answer(text):
    # find right most boxed answer
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]
    # get content of boxed
    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer

## Majority vote (select the most occuring answer)

In [15]:
# Define the majority voting function to get the most common answer
def majority_vote(answers):
    answers = [answer for answer in answers if answer is not None]

    if not answers:
        return None
    # count the occurence of each answer
    counts = {}
    for answer in answers:
        if answer in counts:
            counts[answer] += 1
        else:
            counts[answer] = 1

    max_answer = None
    max_count = 0
    
    for answer, count in counts.items():
        if count > max_count:
            max_answer = answer
            max_count = count
    
    return max_answer

# TIR Agent

In [16]:
class TIRAgent:
    def __init__(self, problem_id, id, problem, tokenizer, max_depth, log):
        # problem id
        self.problem_id = problem_id
        # id of the agent
        self.id = id
        # number of LLM turns
        self.depth = 1
        # maximum number of turns allowed
        self.max_depth = max_depth
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # Problem statement
        self.problem = problem
        # Chat Messages
        self.messages = [{"role": "system", 
                "content": f"""You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
                Your job is to solve math problems which have non-negative integer answers by following the steps: 
                1. Break this problem into smaller subproblems and solve each subproblem step-by-step. 
                2. You will solve most of the problem yourself but you can use python to perform big calculations only like a calculator.And you will tell the user to run the script.
                3. Then using the result from the user, solve rest of the problem and put final answer (only the integer) within \\boxed{{}} 
                """},
            {
                "role": "user", 
                "content": f"""
{self.problem}."""
            }
        ]
        # Last response from the LLM
        self.last_response = None
        # Code blocks from the last response
        self.blocks = []
        # Answers that the LLM generated in \boxed{}
        self.answers = []
        # No python code generated in last response or max_depth reached
        self.is_complete = False
        # File to log answers
        self.log = log
        # Next prompt to the LLM
        self.next_prompt = None
        
    def complete(self):
        # is the Agent done
        return self.is_complete
    
    def add_response(self, response, executor):
        self.depth += 1
        # remember this response
        self.last_response = response
        # add this to the messages history
        self.messages.append({"role": "assistant", "content": response})
        # extract python blocks
        self.blocks = find_python_blocks(response)
        # extract answer from the generated text, if present
        answer = extract_answer(response)
        if answer is not None:
            self.answers.append(answer)
        # is it done?
        self.is_complete = not self._should_continue()
        # if not, use the python executor to create next prompt
        if not self.is_complete:
            self.next_prompt = self._next_prompt(executor) 
            self.messages.append({"role": "user", "content": self.next_prompt})
    
    def _should_continue(self):        
        # quit if max_depth number of turns reached
        if self.depth >= self.max_depth:
            return False
        # if no python code generated, we can stop now
        elif len(self.blocks) > 0:
            return True
        return False
    
    def _next_prompt(self, executor):
        assert not self.is_complete
        assert len(self.blocks) > 0
        # get code result from python execution
        output, status = execute(executor, self.blocks[-1])
        
        prompt = ''
        # if code succeeds give the output
        if status:
            prompt = f"""The python code you provided gives the following output:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        # if code fails, give the error
        else:
            prompt = f"""The python code you provided gives the following error:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        return prompt
    
    
    def next_message(self):
        assert not self.is_complete 
        # apply chat template to get the text
        text = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        return text
        
    
    def final_answer(self):
        # if there no answers yet, we have to return None
        ans = None
        # otherwise return the latest answer
        if len(self.answers) > 0:
            ans = self.answers[-1]
        # log to file
        if self.log:
            self.log.writerow([self.problem_id, self.id, ans])
        # try to convert to integer
        try:
            ans = int(ans)
        except:
            ans = None
        
        return ans        

# Sc-TIR Agent

In [17]:
class SCTIRAgent:
    def __init__(self, problem_id, problem, tokenizer, samples, max_depth, log):
        # problem id
        self.problem_id = problem_id
        # problem statement
        self.problem = problem
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # number of TIRAgents to create
        self.samples = samples
        # maximum number of turns
        self.max_depth = max_depth
        # TIR Agents
        self.agents = [TIRAgent(problem_id, i, problem, tokenizer, max_depth, log) for i in range(samples)]
        # log file
        self.log = log
    
    def complete(self):
        # only complete when all agents are done
        for agent in self.agents:
            if not agent.complete():
                return False
        return True
        
    def get_ready_agents(self):
        # return agents that are not complete yet
        ready_agents = []
        for agent in self.agents:
            if not agent.complete():
                ready_agents.append(agent)
        return ready_agents
    
    def final_answer(self):
        # majority vote agent answers
        assert self.complete()
        answers = [agent.final_answer() for agent in self.agents]
        print(self.problem_id)
        print(answers)
        answer = majority_vote(answers)
        print(answer)
        if answer is None:
            return 0
        return answer

# Load Test Set

In [18]:
import pandas as pd
from vllm import SamplingParams
trans_df=pd.read_csv("/kaggle/input/dlsprint3/test.csv")
sampling_paramsT= SamplingParams(n=1, temperature=0.5, top_p=0.8, max_tokens=5120)

def predict(problem):
    prompt = f" Problem: {problem}.English:"
    messages =[
                {"role": "system", "content": '''You are Qwen, created by Alibaba Cloud. You are a helpful assistant.Your job is to translate a given mathemtaical problem in Bengali to English by following steps.
                1.perform a translation of each sentence
                2.then make a full translation and put it in \\boxed{{}}
                 '''},
                {"role": "user", "content": prompt}
            ]
    text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

        # Generate translations
    # Generate translations
    outputs = llm.generate([text], sampling_paramsT)

    # Collect and process generated outputs
    all_generated_texts = [out.text for out in outputs[0].outputs]

    # Return the first translation result (or process as needed)
    return all_generated_texts[0]

In [19]:
#ex=(predict(trans_df.Problem[1]))
#pr=extract_answer(ex)
#ex,pr

In [20]:
results = []

# Iterate through each row in the test data
for idx, row in trans_df.iterrows():
         
         problem = row['Problem']  # Extract the problem text
         trans = predict(problem)  # Get the prediction using the model
         trans=extract_answer(trans)
         print("Problem: ",idx)
         print("Problem: ",trans)
         results.append({"ID": idx,  
                        "problem": trans
                                })

# Create a DataFrame with the results
sub_df = pd.DataFrame(results)

# Save the results to a CSV file
submission_file_path = '/kaggle/working/trans.csv'  # Define where to save the submission file
sub_df.to_csv(submission_file_path, index=False)

print(f"Submission saved to {submission_file_path}")

Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.50s/it]


Problem:  0
Problem:  How many maximum parts can a cake be divided into by cutting it linearly 2 times?


Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.72s/it]


Problem:  1
Problem:  A pond has 100 stones placed on it. First, a frog jumps over the pond using the 1st, 2nd, 3rd,..., 99th, 100th stones. The second frog jumps over the pond using the 2nd, 4th, 6th,..., 98th, 100th stones. The third frog jumps over the pond using the 3rd, 6th, 9th,..., 99th stones. 100 frogs jumped in this manner. How many stones did an odd number of frogs jump on?


Processed prompts: 100%|██████████| 1/1 [00:21<00:00, 21.91s/it]


...

Problem:  97
Problem:  How many positive integers are there that are factors of at least one of the numbers $12^{12}, 14^{12}$, and $18^9$?


Processed prompts: 100%|██████████| 1/1 [00:11<00:00, 11.95s/it]


Problem:  98
Problem:  The sum of 3 consecutive whole numbers is 216, what is the smallest number?


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.54s/it]

Problem:  99
Problem:  A magician gave you a certain number of pencils on the first day. After that, the magician gives you twice the number of pencils given the previous day every day. You noticed on the fourth day that you have 60 pencils, so how many pencils did the magician give you on the first day?
Submission saved to /kaggle/working/trans.csv


In [21]:
import pandas as pd
test_df = pd.read_csv('/kaggle/working/trans.csv')
#test_df['ID'] = range(len(test_df))

## Load 5 problems since we are short on time

In [22]:
test_df

,ID,problem
0,0,How many maximum parts can a cake be divided i...
1,1,"A pond has 100 stones placed on it. First, a f..."
2,2,"Let $f(x) = x^{67-x^{67-x^{67-\dots}}}$, where..."
3,3,Samin \ and \ Swarga \ use \ only \ 0 \ and \ ...
4,4,"A, B, C are three stands. To move 2 disks from..."
...,...,...
95,95,A box has some red and blue balls. If one more...
96,96,A four-digit whole number is divisible by 11 w...
97,97,How many positive integers are there that are ...
98,98,"The sum of 3 consecutive whole numbers is 216,..."


In [23]:
if DEBUG:
    test_df = test_df[:5]
    torch.cuda.empty_cache()
    gc.collect()

# Configure LLM and Python REPL

In [24]:
sampling_params = vllm.SamplingParams(max_tokens=10500, temperature=.35, top_p=.775)
executor = PythonREPL()

# Run the Agents

## TIR Agent

In [25]:
'''for row in test_df.values[:2]:
    problem_id = row[0]
    problem = row[1]
    
    agent = TIRAgent(problem_id, 0, problem, tokenizer, max_depth=4, log=None)
    
    while not agent.complete():
        text = agent.next_message()
        # get response from LLM
        response = llm.generate([text], sampling_params)
        # pass in python executor, since response might contain python code
        agent.add_response(response[0].outputs[0].text, executor)
    
    for message in agent.messages:
        print(f"Role: {message['role']}\n")
        print(f"Content:\n {message['content']}\n")
    
    answer = agent.final_answer()
    print(f"Final answer: {answer}")'''
    

'for row in test_df.values[:2]:\n    problem_id = row[0]\n    problem = row[1]\n    \n    agent = TIRAgent(problem_id, 0, problem, tokenizer, max_depth=4, log=None)\n    \n    while not agent.complete():\n        text = agent.next_message()\n        # get response from LLM\n        response = llm.generate([text], sampling_params)\n        # pass in python executor, since response might contain python code\n        agent.add_response(response[0].outputs[0].text, executor)\n    \n    for message in agent.messages:\n        print(f"Role: {message[\'role\']}\n")\n        print(f"Content:\n {message[\'content\']}\n")\n    \n    answer = agent.final_answer()\n    print(f"Final answer: {answer}")'

## SC-TIR Agent

In [26]:
'''for row in trans_df.values[85:86]:
    problem_id = row[0]
    problem = "A pond has 100 stones placed on it. First, a frog jumps over the pond using the 1st, 2nd, 3rd,..., 99th, 100th stones. The second frog jumps over the pond using the 2nd, 4th, 6th,..., 98th, 100th stones. The third frog jumps over the pond using the 3rd, 6th, 9th,..., 99th stones. 100 frogs jumped in this manner. How many stones did an odd number of frogs jump on?"
    
    agent = SCTIRAgent(problem_id, problem, tokenizer, samples=4, max_depth=4, log=None)
    
    while not agent.complete():
        ready_agents = agent.get_ready_agents()
        texts = [a.next_message() for a in ready_agents]
        # get response from LLM
        responses = llm.generate(texts, sampling_params)
        # pass response to the agents
        for i, ready_agent in enumerate(ready_agents):
            print(responses[i].outputs[0].text)
            ready_agent.add_response(responses[i].outputs[0].text, executor)
    
    answer = agent.final_answer()
    print(f"Problem: {problem}")
    print(f"Final answer: {answer}")'''

'for row in trans_df.values[85:86]:\n    problem_id = row[0]\n    problem = "A pond has 100 stones placed on it. First, a frog jumps over the pond using the 1st, 2nd, 3rd,..., 99th, 100th stones. The second frog jumps over the pond using the 2nd, 4th, 6th,..., 98th, 100th stones. The third frog jumps over the pond using the 3rd, 6th, 9th,..., 99th stones. 100 frogs jumped in this manner. How many stones did an odd number of frogs jump on?"\n    \n    agent = SCTIRAgent(problem_id, problem, tokenizer, samples=4, max_depth=4, log=None)\n    \n    while not agent.complete():\n        ready_agents = agent.get_ready_agents()\n        texts = [a.next_message() for a in ready_agents]\n        # get response from LLM\n        responses = llm.generate(texts, sampling_params)\n        # pass response to the agents\n        for i, ready_agent in enumerate(ready_agents):\n            print(responses[i].outputs[0].text)\n            ready_agent.add_response(responses[i].outputs[0].text, executor)\n

# Run Inference

# Create submission

## Also log agent answers

In [27]:
file = open('submission1.csv', 'w', encoding='utf-8')
log_file = open('log.csv', 'w', encoding='utf-8')

submission = csv.writer(file)
log = csv.writer(log_file)

submission.writerow(['ID', 'Answer'])
log.writerow(['ID', "Agent ID", 'Answer'])

20

## Configure LLM sampling parameters and Python REPL

## Use a queue to Batch inference

In [28]:
%%time

boxed_answers = {}
agents = []

q = Queue()

iterator = iter(tqdm(test_df.values))

while True:
    for agent in agents:
        if agent.complete():
            boxed_answers[agent.problem_id] = agent.final_answer()

    agents[:] = list(filter(lambda a: not a.complete(), agents))

    while q.qsize() < BATCH_SIZE:
        try:
            row = next(iterator)
        except StopIteration:
            break

        id = row[0]
        problem = row[1]

        agent = SCTIRAgent(id, problem, tokenizer, K, DEPTH, log)
        
        agents.append(agent)

        for tir_agent in agent.get_ready_agents():
            q.put_nowait(tir_agent)
            
    if q.empty():
        break
        
    
    ready_agents = []
    texts = []
    for _ in range(BATCH_SIZE):
        try:
            agent = q.get_nowait()
            ready_agents.append(agent)
            texts.append(agent.next_message())
        except:
            break

    
    responses = llm.generate(texts, sampling_params)
    responses = [response.outputs[0].text for response in responses]
    
    for i in range(len(ready_agents)):
        agent = ready_agents[i]
        response = responses[i]
        agent.add_response(response, executor)
        if not agent.complete():
            q.put_nowait(agent)
   

 16%|█▌        | 16/100 [06:33<34:27, 24.62s/it]

1
[10, 10, 10, 10]
10
2
[4, 4, 4, 4]
4
5
[0, 0, 0, 0]
0
15
[240, 240, 260, 240]
240



 26%|██▌       | 26/100 [14:35<44:04, 35.74s/it]

0
[4, 4, 4, 4]
4
4
[1023, 1023, 1023, 1023]
1023
6
[11, 11, 11, 2]
11
7
[207, 327, 321, 200]
207
8
[14, 14, 14, 14]
14
9
[63, 60, 60, 63]
63
10
[2, 2, 2, 2]
2
12
[1901, 2001, 1901, 1901]
1901
13
[1000, 1000, 1000, 1000]
1000
20
[97, 97, 85, 97]
97
22
[15, 15, 15, 15]
15
24
[145, 145, 145, 145]
145



 37%|███▋      | 37/100 [37:49<1:18:11, 74.46s/it]

...

WARNING 10-31 15:25:54 scheduler.py:245] Input prompt (10073 tokens) is too long and exceeds limit of 10000


Processed prompts: 100%|██████████| 49/49 [04:41<00:00,  5.74s/it]


74
[16, 16, 16, 16]
16
76
[120, 120, 288, 192]
120
79
[197, 197, 1197, 197]
197
82
[7, 7, 7, 7]
7
83
[255, 16, 255, 16]
255
84
[24, 24, 24, 24]
24
88
[720, 720, 720, 720]
720
94
[1012, 1012, 1012, 1012]
1012
95
[9, 9, 9, 9]
9


Processed prompts: 100%|██████████| 16/16 [01:35<00:00,  5.97s/it]

85
[3, 3, 1, 3]
3
87
[25, 25, 50, 25]
25
89
[None, 45, 135, 135]
135
93
[24, 24, 24, 24]
24
96
[94094, 18000, 78474, 78474]
78474
97
[674, 6175, 561, 531]
674
98
[71, 71, 71, 71]
71
99
[4, 4, 4, 7]
4
CPU times: user 1h 25min 10s, sys: 41.2 s, total: 1h 25min 52s
Wall time: 1h 27min 12s


## Write to submission file

In [29]:
for id, answer in boxed_answers.items():
    submission.writerow([id, answer])

## Close files

In [30]:

file.close()
log_file.close()
import pandas as pd
df=pd.read_csv("/kaggle/working/submission1.csv")
df.set_index('ID',inplace=True)
df.sort_index(inplace=True)
df.reset_index(inplace=True)
df.to_csv('submission.csv',index=False)